### What does explainable mean?
Deep neural networks are quite successful in many use-cases, but these models can be hard to debug and to understand what’s going on. Our aim is to understand how much certain words influence the prediction of our named entity tagger. We want a human-understandable qualitative explanation which enables an interpretation of the underlying algorithm.

### Data preperation

In [2]:
import os
from urllib.request import urlretrieve
import zipfile
import glob

if not os.path.exists('data'):
    os.makedirs('data')
    
# Download data
url ='https://storage.googleapis.com/kaggle-datasets/1014/4361/entity-annotated-corpus.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1568009584&Signature=hSdo87OA6a0gwPDkkK1eHg1l3bfLg%2FO9CTZ%2Fma6B%2F%2BmcwwU7OQiEmjKpgJ8ROWbPXrwjhED3u3dkas63MRbL1Rin3XUeWKU3y6TqgK%2FmleA3SVf6jBqXTOfRjyDaPXPNYdJLYFCWIDbygZPxoNEmXel3ZV%2B3MQgDOKH%2FzAP1NLuU5y6VHaFePdsruHAb1KICRY6qvsl5gFTYyBkJw3xO0qoF8oNkG3C4uUDaTEaqVK7FOfAw7OkkpTXqc9GtjUdsI3Dr11QNYgTmIOdreqk0fgr89QaenXBTfZlS8hqMu46Ik1VrX0Y5zfOSH7Rd3T5ltDvNNANlh%2FA%2BpJr0y16cHA%3D%3D'

urlretrieve(url, 'data/kaggle_ner.zip')

with zipfile.ZipFile('data/kaggle_ner.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')
    
import glob

glob.glob('data/*')

['data/kaggle_ner.zip', 'data/ner.csv', 'data/ner_dataset.csv']

In [3]:
import pandas as pd    
import numpy as np
from collections import Counter

data = pd.read_csv("data/ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")

class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
        
getter = SentenceGetter(data)
sentences = getter.sentences

max_len = 50
max_len_char = 10

words = list(set(data["Word"].values))
n_words = len(words)

tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags
tag2idx = {t: i for i, t in enumerate(tags)}

labels = [[s[2] for s in sent] for sent in sentences]
sentences = [" ".join([s[0] for s in sent]) for sent in sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [0]:
word_cnt = Counter(data["Word"].values)
vocabulary = set(w[0] for w in word_cnt.most_common(5000))

word2idx = {"PAD": 0, "UNK": 1}
word2idx.update({w: i for i, w in enumerate(words) if w in vocabulary})

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


X = [[word2idx.get(w, word2idx["UNK"]) for w in s.split()] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=word2idx["PAD"])

y = [[tag2idx[l_i] for l_i in l] for l in labels]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [0]:
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, shuffle=False)

### Setup The NER Model
We use the simple LSTM model. But the procedure shown here applies to all kinds of sequence models.

In [9]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D, Lambda
from tensorflow.keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D, add

word_input = Input(shape=(max_len,))

# Instantiate the custom Bert Layer defined above
embedding = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(word_input)

x = Bidirectional(LSTM(units=256, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)

out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)


model = Model(word_input, out)

adam = tf.keras.optimizers.Adam(clipnorm = 1.)
model.compile(optimizer=adam, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            1758900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 512)           628736    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 17)            8721      
Total params: 2,396,357
Trainable params: 2,396,357
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(X_tr, y_tr.reshape(*y_tr.shape, 1),
                    batch_size=32, epochs=5,
                    validation_split=0.1, verbose=1)

Train on 38846 samples, validate on 4317 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
38846/38846 [==============================] - 444s 11ms/sample - loss: 0.1584 - acc: 0.9612 - val_loss: 0.0704 - val_acc: 0.9795
Epoch 2/5
38846/38846 [==============================] - 440s 11ms/sample - loss: 0.0634 - acc: 0.9808 - val_loss: 0.0585 - val_acc: 0.9826
Epoch 3/5
38846/38846 [==============================] - 448s 12ms/sample - loss: 0.0550 - acc: 0.9828 - val_loss: 0.0562 - val_acc: 0.9829
Epoch 4/5
38846/38846 [==============================] - 446s 11ms/sample - loss: 0.0506 - acc: 0.9839 - val_loss: 0.0539 - val_acc: 0.9835
Epoch 5/5
38846/38846 [==============================] - 442s 11ms/sample - loss: 0.0477 - acc: 0.9847 - val_loss: 0.0531 - val_acc: 0.9839


### Now Look At The Predictions And Explain Them

To explain the predictions, we use the LIME algorithm implemented in the eli5 library. We assume you already now what the algorithm is doing. You can read more about it in [this post](https://www.depends-on-the-definition.com/debugging-black-box-text-classifiers-with-lime/).

In [12]:
!pip install eli5
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler

     |████████████████████████████████| 112kB 3.5MB/s 


Using TensorFlow backend.


Now we create a small python class, that holds our preprocessing and prediction of the model. To apply LIME we just need a function to make predictions on texts. We use the closure pattern in `get_predict_function` which returns a function that takes a list of texts, processes them and returns the predictions of our previously trained model.

### The trick
To make the LIME algorithm work for us, we need to rephrase our problem as a simple multi-class classification problem. We do this by selecting before-hand for which word we want to explain the prediction. This is done by passing the `word_index` to the `get_predict_function` method.

In [0]:
class NERExplainerGenerator(object):
    
    def __init__(self, model, word2idx, tag2idx, max_len):
        self.model = model
        self.word2idx = word2idx
        self.tag2idx = tag2idx
        self.idx2tag = {v: k for k,v in tag2idx.items()}
        self.max_len = max_len
        
    def _preprocess(self, texts):
        X = [[self.word2idx.get(w, self.word2idx["UNK"]) for w in t.split()]
             for t in texts]
        X = pad_sequences(maxlen=self.max_len, sequences=X,
                          padding="post", value=self.word2idx["PAD"])
        return X
    
    def get_predict_function(self, word_index):
        def predict_func(texts):
            X = self._preprocess(texts)
            p = self.model.predict(X)
            return p[:,word_index,:]
        return predict_func

Let’s have a look at some interesting samples. For example the 46781th text in our data set.

In [14]:
index = 46781
label = labels[index]
text = sentences[index]
print(text)
print()
print(" ".join([f"{t} ({l})" for t, l in zip(text.split(), label)]))

Nigeria 's President Olusegun Obasanjo expressed his condolences , noting the late pontiff promoted religious tolerance and democracy in the West African nation .

Nigeria (B-geo) 's (O) President (B-per) Olusegun (I-per) Obasanjo (I-per) expressed (O) his (O) condolences (O) , (O) noting (O) the (O) late (O) pontiff (O) promoted (O) religious (O) tolerance (O) and (O) democracy (O) in (O) the (O) West (O) African (B-gpe) nation (O) . (O)


Now start to explain the prediction. We first initialize our generator object.

In [0]:
explainer_generator = NERExplainerGenerator(model, word2idx, tag2idx, max_len)

We want to explain the NER prediction for the word “Obasanjo”, so we pick word_index=4 and generate the respective prediction function.

In [0]:
word_index = 4
predict_func = explainer_generator.get_predict_function(word_index=word_index)

Here we have to specify a sampler for the LIME algorithm. This controls how the algorithm samples perturbed samples from the text we want to explain. Read more about this in this article or the eli5 documentation.

In [18]:
sampler = MaskingTextSampler(
    replacement="UNK",
    max_replace=0.7,
    token_pattern=None,
    bow=False
)

samples, similarity = sampler.sample_near(text, n_samples=4)
print(samples)

("Nigeria 'UNK President Olusegun Obasanjo expressed his condolences , noting the late pontiff promoted UNK UNK and democracy in the West African UNK .", "Nigeria 's President Olusegun Obasanjo expressed his condolences , noting the late UNK promoted religious tolerance and UNK in UNK West African nation .", "Nigeria 's UNK UNK UNK expressed UNK UNK , UNK UNK late pontiff promoted religious tolerance UNK democracy UNK UNK UNK UNK nation .", "UNK 'UNK UNK Olusegun UNK expressed his condolences , noting UNK late pontiff UNK religious tolerance and UNK in UNK UNK African nation .")


Finally, we set up the `TextExplainer` and explain the prediction.

In [19]:
te = TextExplainer(
    sampler=sampler,
    position_dependent=True,
    random_state=42
)

te.fit(text, predict_func)

te.explain_prediction(
    target_names=list(explainer_generator.idx2tag.values()),
    top_targets=3
)

Explanation(estimator="SGDClassifier(alpha=0.001, average=False, class_weight=None,\n              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,\n              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,\n              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',\n              power_t=0.5,\n              random_state=<mtrand.RandomState object at 0x7f737dafa120>,\n              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,\n              warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='I-per', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[2] President', weight=2.86761644633491, std=None, value=1.0), FeatureWeight(feature='[5] expressed', weight=1.1420026690620475, std=None, value=1.0), FeatureWeight(feature='[4] Obasanjo', weight=0.6733353904785359, std=None, value=1.0), FeatureWeight(feature='[6] his', weight=0.18337585616181237, std=None, value=1.0), FeatureWeight(feature='[7] condolences', weight=0.08561203718529581, std=None, value=1.0), FeatureWeight(feature='[17] in', weight=0.043560976273456065, std=None, value=1.0), FeatureWeight(feature='[15] and', weight=0.04015168839885265, std=None, value=1.0), FeatureWeight(feature='[3] Olusegun', weight=0.03139560848927649, std=None, value=1.0), FeatureWeight(feature='[12] promoted', weight=0.013656223142549632, std=None, value=1.0)], neg=[FeatureWeight(feature='[1] s', weight=-0.6083099919086711, std=None, value=1.0), FeatureWeight(feature='[0] Nigeria', weight=-0.47144577653237696, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.3752031516646971, std=None, value=1.0), FeatureWeight(feature='[11] pontiff', weight=-0.04760956973257726, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.96927705807074, score=3.578138405688414, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document="Nigeria 's President Olusegun Obasanjo expressed his condolences , noting the late pontiff promoted religious tolerance and democracy in the West African nation .", spans=[('Nigeria', [(0, 7)], -0.47144577653237696), ('s', [(9, 10)], -0.6083099919086711), ('President', [(11, 20)], 2.86761644633491), ('Olusegun', [(21, 29)], 0.03139560848927649), ('Obasanjo', [(30, 38)], 0.6733353904785359), ('expressed', [(39, 48)], 1.1420026690620475), ('his', [(49, 52)], 0.18337585616181237), ('condolences', [(53, 64)], 0.08561203718529581), ('pontiff', [(83, 90)], -0.04760956973257726), ('promoted', [(91, 99)], 0.013656223142549632), ('and', [(120, 123)], 0.04015168839885265), ('in', [(134, 136)], 0.043560976273456065)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=3.953341557353111, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.3752031516646971, std=None, value=1.0)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='I-org', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[0] Nigeria', weight=0.523593327624631, std=None, value=1.0), FeatureWeight(feature='[4] Obasanjo', weight=0.3535456411072404, std=None, value=1.0), FeatureWeight(feature='[1] s', weight=0.2927320923886197, std=None, value=1.0)], neg=[FeatureWeight(feature='[2] President', weight=-2.9828843041065554, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-1.4481618502090499, std=None, value=1.0), FeatureWeight(feature='[5] expressed', weight=-0.47134344120401467, std=None, value=1.0), FeatureWeight(feature='[6] his', weight=-0.2862690240968326, std=None, value=1.0), FeatureWeight(feature='[19] West', weight=-0.0967680656945793, std=None, value=1.0), FeatureWeight(feature='[7] condolences', weight=-0.09247106009783232, std=None, value=1.0), FeatureWeight(feature='[3] Olusegun', weight=-0.07851904310991183, std=None, value=1.0), FeatureWeight(feature='[18] the', weight=-0

Very nice! As expected, the model predicted I-per for a later part of a person name. The word President is a strong indicator that the following word is part of a name. This indicates, that in the dataset, President is often part of the annotation of a Person.